In [422]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import KFold
import gc

import warnings
warnings.simplefilter('ignore')

In [423]:
train = pd.read_csv('./data/train_dataset.csv')
test = pd.read_csv('./data/evaluation_public.csv')
data = pd.concat([train, test]).reset_index(drop=True)

data['isnull'] = data.iloc[:,1:].isnull().mean(axis = 1)
data['isnull'] = data.iloc[:,1:].isnull().mean(axis = 1)
data['time'] = pd.to_datetime(data['time'])
data['hour'] = data['time'].dt.hour
data['minute'] = data['time'].dt.minute
data['weekday'] = data['time'].dt.weekday
data['ts'] = data['hour']*3600 + data['minute']*60 + data['time'].dt.second

In [424]:
train = data.iloc[:140480].reset_index(drop = True)
test = data.iloc[140480:].reset_index(drop = True)

## 通过特征重要性、对抗验证、特征相关性，剔除了这7个特征，可能剔除错了也可能没剔除干净
feas = [f for f in train.columns if f not in ['time', 'Label1', 'Label2','Label1_log','Label2_log','CS_LL',
                                              'B_QY_ORP','JS_TN', 'CS_SW','MCCS_NH4','N_HYC_JS_DO','MCCS_NO3','JS_SW',
                                             ]]
print(f"feature len is {len(feas)}")
for f in feas:
    train[f] = train[f].fillna(method='ffill')
train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
test['is_train'] = False
train['is_train'] = True
# data = pd.concat([train, test]).reset_index(drop=True)

feature len is 23


In [425]:
print(feas)

['JS_NH3', 'CS_NH3', 'CS_TN', 'JS_LL', 'JS_COD', 'CS_COD', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP', 'isnull', 'hour', 'minute', 'weekday', 'ts']


In [426]:
# df_list = []
# i = 0
# add_featuers = []
# length = 0
# for f in ['JS_NH3', 'CS_NH3', 'CS_TN', 'JS_LL', 'JS_COD', 'CS_COD', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL',
#           'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP']:
#     for r in [3,5,7,10]:
#         train_rolling = data[f].rolling(window=r, center=True)
#         f_mean_name = 'rolling{}_{}_mean'.format(r,f)
#         f_max_name = 'rolling{}_{}_max'.format(r,f)
#         f_min_name = 'rolling{}_{}_min'.format(r,f)
#         f_std_name = 'rolling{}_{}_std'.format(r,f)
#         data[f_mean_name] = train_rolling.mean().fillna(0).values
#         data[f_max_name] = train_rolling.max().fillna(0).values
#         data[f_min_name] = train_rolling.min().fillna(0).values
#         data[f_std_name] = train_rolling.std().fillna(0).values
#         data[f'{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
#         data[f'{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
#         data[f'{f}_{f_min_name}_cha'] = data[f] - data[f_min_name]
#         if i == 0:
#             add_featuers.append(f_mean_name)
#             add_featuers.append(f_max_name)
#             add_featuers.append(f_min_name)
#             add_featuers.append(f_std_name)
#             add_featuers.append(f'{f}_{f_mean_name}_cha')
#             add_featuers.append(f'{f}_{f_max_name}_cha')
#             add_featuers.append(f'{f}_{f_min_name}_cha')
# feas.extend(add_featuers)
# train = data[data['is_train']].reset_index(drop=True)
# test = data[~data['is_train']].reset_index(drop=True)

train['Label1'] = np.log1p(train['Label1'])
train['Label2'] = np.log1p(train['Label2'])

In [427]:
len(train)

35068

In [439]:
from sklearn.metrics import mean_squared_error

def train_model(data_, test_, y_, folds_,features):
    oof_preds = np.zeros(data_.shape[0])
    sub_preds = np.zeros(test_.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = features
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(data_)):
        trn_x, trn_y = data_[feats].iloc[trn_idx], y_.iloc[trn_idx]
        val_x, val_y = data_[feats].iloc[val_idx], y_.iloc[val_idx]
        clf = lgb.LGBMRegressor(
                # boosting="gbdt",
                num_leaves=2**4,
                learning_rate=0.03,
                n_estimators=40000,
                min_child_weight=1,
                min_data_in_leaf=10,
                subsample = 0.7,
                feature_fraction=0.6,
                bagging_seed=1,
                reg_alpha=1,
                reg_lambda=1,  # 此处不改了
                min_sum_hessian_in_leaf=0.01,
                random_state=1212
            )

        clf.fit(trn_x, trn_y,
                eval_set=[(trn_x, trn_y), (val_x, val_y)],
                eval_metric='mse', verbose=500, early_stopping_rounds=40  # 30
                )

        oof_preds[val_idx] = clf.predict(val_x, num_iteration=clf.best_iteration_)
        sub_preds += clf.predict(test_[feats], num_iteration=clf.best_iteration_) / folds_.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        print('Fold %2d rmse : %.6f' % (n_fold + 1, np.sqrt(mean_squared_error(np.expm1(val_y), np.expm1(oof_preds[val_idx])))))
        del clf, trn_x, trn_y, val_x, val_y
        gc.collect()
    sv_score = np.sqrt(mean_squared_error(np.expm1(y_), np.expm1(oof_preds)))
    feature_importance_df = feature_importance_df.groupby(['feature'])[['importance']].mean().reset_index(drop=False)
    feature_importance_df.columns = ['feature', 'importance']
    print('Full rmse score %.6f' % sv_score)

    test_['isDefault'] = np.expm1(sub_preds)

    return oof_preds, test_['isDefault'], sv_score, feature_importance_df


In [440]:
folds = 5
kf = KFold(n_splits=folds, shuffle=True, random_state=2222)
scores = []
feature_score_list = []
for label in ['Label1', 'Label2']:
    _, test[label], score, feature_score = train_model(train, test, train[label], kf, feas)
    scores.append(score)
    feature_score_list.append(feature_score)
last_score = 1/(1+(scores[0]+scores[1])/2)*1000
print(f"last_score = {last_score}")

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.01, min_child_weight=1 will be ignored. Current value: min_sum_hessian_in_leaf=0.01
Training until validation scores don't improve for 40 rounds
[500]	training's l2: 0.0101742	valid_1's l2: 0.0142672
[1000]	training's l2: 0.00758818	valid_1's l2: 0.0130543
[1500]	training's l2: 0.00625884	valid_1's l2: 0.0124427
[2000]	training's l2: 0.00534805	valid_1's l2: 0.0120289
[2500]	training's l2: 0.00470987	valid_1's l2: 0.0117421
[3000]	training's l2: 0.00423333	valid_1's l2: 0.0115188
[3500]	training's l2: 0.00384468	valid_1's l2: 0.0113664
[4000]	training's l2: 0.00354411	valid_1's l2: 0.0112476
[4500]	training's l2: 0.00328021	valid_1's l2: 0.0111536
Early stopping, best iteration is:
[4958]	tr

In [430]:
# print(f"last_score = {last_score}")

# 由于线上线下分数差别太大，查看是不是部分特征分布抬不均衡

In [433]:
# train['test'] = 0
# test['test'] = 1
# all_data_v2 = pd.concat([train, test]).reset_index(drop=True)
# print(len(all_data_v2))
#
# X_train, X_test, y_train, y_test  = train_test_split(all_data_v2[test_feature].copy(deep=True), all_data_v2['test'].copy(deep=True), test_size=0.2)
# fortest_model = lgb.LGBMClassifier(
#         num_leaves=2**4,
#         learning_rate=0.03,
#         n_estimators=300,
#         min_child_weight=1,
#         min_data_in_leaf=10,
#         subsample = 0.7,
#         feature_fraction=0.4,
#         bagging_seed=1,
#         reg_alpha=1,
#         reg_lambda=1,  # 此处不改了
#         min_sum_hessian_in_leaf=0.01,
#         random_state=1212)
#
# fortest_model.fit(X_train, y_train,
#                 eval_set=[(X_train, y_train), (X_test, y_test)],
#                 eval_metric='auc', verbose=100, early_stopping_rounds=40  # 30
#                 )
# test_feture = pd.DataFrame()
# test_feture['feature'] = test_feature
# test_feture['score'] = fortest_model.feature_importances_
# test_feture.sort_values(by=['score'], ascending=False, inplace=True, ignore_index=True)

In [434]:
# test_feture # rolling3_CS_NH3_std  rolling5_CS_NH3_std

In [435]:
# xxxx = list(test_feture['feature'][-150:])

In [436]:
# folds = 5
# kf = KFold(n_splits=folds, shuffle=True, random_state=2222)
# scores = []
# for label in ['Label1', 'Label2']:
#     _, test[label], score, feature_score = train_model(train, test, train[label], kf, test_feture['feature'].values[-150:])
#     scores.append(score)
# last_score = 1000/(1+(scores[0]+scores[1])/2)
# print(f"last_score = {last_score}")

In [437]:
sample_submission = pd.read_csv("./data/sample_submission.csv")
sample_submission['Label1'] = test['Label1']
sample_submission['Label2'] = test['Label2']
sample_submission.to_csv('./data/base_line.csv', index=False)

In [438]:
print(list(test_feture['feature']))

['rolling7_B_CS_MQ_SSLL_min', 'rolling7_B_CS_MQ_SSLL_max', 'JS_LL_rolling10_JS_LL_max_cha', 'N_HYC_DO_rolling5_N_HYC_DO_min_cha', 'N_CS_MQ_SSLL']
